In [33]:
#try elastic transform https://www.kaggle.com/babbler/mnist-data-augmentation-with-elastic-distortion
import numpy as np
from PIL import Image
from IPython.display import display
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage.interpolation import map_coordinates
import cv2


def elastic_transform(image, alpha_range, sigma, random_state=None):
    """Elastic deformation of images as described in [Simard2003]_.
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
       Convolutional Neural Networks applied to Visual Document Analysis", in
       Proc. of the International Conference on Document Analysis and
       Recognition, 2003.
       
   # Arguments
       image: Numpy array with shape (height, width, channels). 
       alpha_range: Float for fixed value or [lower, upper] for random value from uniform distribution.
           Controls intensity of deformation.
       sigma: Float, sigma of gaussian filter that smooths the displacement fields.
       random_state: `numpy.random.RandomState` object for generating displacement fields.
    """
    
    if random_state is None:
        random_state = np.random.RandomState(None)
        
    if np.isscalar(alpha_range):
        alpha = alpha_range
    else:
        alpha = np.random.uniform(low=alpha_range[0], high=alpha_range[1])

    shape = image.shape
    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha

    x, y, z = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), np.arange(shape[2]), indexing='ij')
    indices = np.reshape(x+dx, (-1, 1)), np.reshape(y+dy, (-1, 1)), np.reshape(z, (-1, 1))

    return map_coordinates(image, indices, order=1, mode='reflect').reshape(shape)

preprocessing_function=lambda x: elastic_transform(x, alpha_range=[20, 25], sigma=4)

In [34]:
from shutil import copytree
import tensorflow as tf

datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=22,preprocessing_function=preprocessing_function)
#datagen = tf.keras.preprocessing.image.ImageDataGenerator(vertical_flip=True)  

numerals=["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"]
#numerals=['i']

n_samples_target=900
batch_size=32
seed=0

for numeral in numerals:
    main_source_dir=f'../data/train-clean/'
    numeral_source_dir=f'{main_source_dir}/{numeral}'
    save_dir=f'../data/train-aug/{numeral}'
    
    copytree(numeral_source_dir,save_dir,dirs_exist_ok=True)


    img_flow=datagen.flow_from_directory(
        main_source_dir,
        save_to_dir=f'../data/train-aug/{numeral}',
        classes=[numeral],
        #subset='training',
        batch_size=batch_size,
        target_size=(32, 32),
        save_prefix="aug",
        #color_mode="rgb"
        shuffle=False,
        seed=seed,
    )
    n_samples=img_flow.samples

    while n_samples<n_samples_target:
        next(img_flow)
        n_samples+=batch_size
    
    
#
# len(img_flow)



Found 244 images belonging to 1 classes.
Found 139 images belonging to 1 classes.
Found 154 images belonging to 1 classes.
Found 264 images belonging to 1 classes.
Found 181 images belonging to 1 classes.
Found 161 images belonging to 1 classes.
Found 181 images belonging to 1 classes.
Found 173 images belonging to 1 classes.
Found 199 images belonging to 1 classes.
Found 162 images belonging to 1 classes.


In [31]:
next(img_flow)

(array([[[[255., 255., 255.],
          [255., 255., 255.],
          [255., 255., 255.],
          ...,
          [255., 255., 255.],
          [255., 255., 255.],
          [255., 255., 255.]],
 
         [[255., 255., 255.],
          [255., 255., 255.],
          [255., 255., 255.],
          ...,
          [255., 255., 255.],
          [255., 255., 255.],
          [255., 255., 255.]],
 
         [[255., 255., 255.],
          [255., 255., 255.],
          [255., 255., 255.],
          ...,
          [255., 255., 255.],
          [255., 255., 255.],
          [255., 255., 255.]],
 
         ...,
 
         [[255., 255., 255.],
          [255., 255., 255.],
          [255., 255., 255.],
          ...,
          [255., 255., 255.],
          [255., 255., 255.],
          [255., 255., 255.]],
 
         [[255., 255., 255.],
          [255., 255., 255.],
          [255., 255., 255.],
          ...,
          [255., 255., 255.],
          [255., 255., 255.],
          [255., 255., 255.

In [21]:
tf.keras.preprocessing.image.ImageDataGenerator??

Init signature:
tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    dtype=None,
)
Source:        
class ImageDataGenerator(image.ImageDataGenerator):
  """Generate batches of tensor image data with real-time data augmentation.

   The data will be looped over (in batches).

  Arguments:
      featurewise_center: Boolean.
          Set input mean to 0 over the dataset, feature-wise.
      samplewise_center: Boolean. Set each sample mean to 0.
 

In [16]:
img_flow.samples

244

In [ ]:
images, labels = next(img_flow)

In [ ]:
labels

In [ ]:
from PIL import Image
from IPython.display import display


display(Image.fromarray(images))